In [1]:
!pip install pyspark findspark

In [2]:
# Завантаження mysql-connector
!wget -q https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-j-8.0.32.tar.gz
!tar -xzf mysql-connector-j-8.0.32.tar.gz
!cp mysql-connector-j-8.0.32/mysql-connector-j-8.0.32.jar /content/

In [3]:
# Spark сесія

# Встановлення Kafka-конектора для Spark
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.1/spark-sql-kafka-0-10_2.12-3.5.1.jar -P /content/
!wget https://repo1.maven.org/maven2/org/apache/kafka/kafka-clients/3.7.0/kafka-clients-3.7.0.jar -P /content/
!wget https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.12.0/commons-pool2-2.12.0.jar -P /content/

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import random, time
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .config("spark.jars", "/content/spark-sql-kafka-0-10_2.12-3.5.1.jar,"
                          "/content/kafka-clients-3.7.0.jar,"
                          "/content/commons-pool2-2.12.0.jar,"
                          "/content/mysql-connector-j-8.0.32.jar") \
    .getOrCreate()


--2025-04-13 09:42:55--  https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.1/spark-sql-kafka-0-10_2.12-3.5.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432340 (422K) [application/java-archive]
Saving to: ‘/content/spark-sql-kafka-0-10_2.12-3.5.1.jar’

spark-sql-kafka-0-1 100%[===================>] 422.21K   824KB/s    in 0.5s    

2025-04-13 09:42:56 (824 KB/s) - ‘/content/spark-sql-kafka-0-10_2.12-3.5.1.jar’ saved [432340/432340]

--2025-04-13 09:42:56--  https://repo1.maven.org/maven2/org/apache/kafka/kafka-clients/3.7.0/kafka-clients-3.7.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting r

In [4]:
# Під'єднання до MySQL та завантаження даних
jdbc_url = "jdbc:mysql://217.61.57.46:3306/olympic_dataset"
jdbc_table = "athlete_bio"
jdbc_user = "neo_data_admin"
jdbc_password = "Proyahaxuqithab9oplp"

athlete_bio_df = spark.read.format('jdbc').options(
    url=jdbc_url,
    driver='com.mysql.cj.jdbc.Driver',
    dbtable=jdbc_table,
    user=jdbc_user,
    password=jdbc_password
).load()

athlete_bio_df = athlete_bio_df \
    .filter(athlete_bio_df['height'].cast("float").isNotNull() & athlete_bio_df['weight'].cast("float").isNotNull()) \
    .withColumn('height', col('height').cast('float')) \
    .withColumn('weight', col('weight').cast('float'))

athlete_bio_df.show(5)

+----------+----------------+------+--------------+------+------+-----------------+-----------+--------------------+--------------------+
|athlete_id|            name|   sex|          born|height|weight|          country|country_noc|         description|       special_notes|
+----------+----------------+------+--------------+------+------+-----------------+-----------+--------------------+--------------------+
|     65649|    IvankaBonova|Female|    4April1949| 166.0|  55.0|         Bulgaria|        BUL|PersonalBest40053...|                 nan|
|    112510|NataliyaUryadova|Female|   15March1977| 184.0|  70.0|RussianFederation|        RUS|                 nan|ListedinOlympians...|
|    114973|EssaIsmailRashed|  Male|14December1986| 165.0|  55.0|            Qatar|        QAT|PersonalBest10000...|ListedinOlympians...|
|    133041| VincentRiendeau|  Male|13December1996| 178.0|  68.0|           Canada|        CAN|                 nan|ListedinOlympians...|
|    110425|      TanjaMorel|Femal

In [5]:
# Створення стрімінг-джерела даних з Kafka

from pyspark.sql.functions import from_json, col, current_timestamp
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Вказуємо схему для JSON-повідомлень Kafka
schema = StructType([
    StructField("athlete_id", IntegerType(), True),
    StructField("medal", StringType(), True),
    StructField("sport", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("country_noc", StringType(), True),
])

# Створення Streaming DataFrame з Kafka
kafka_stream_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "athlete_event_results") \
    .option("startingOffsets", "earliest") \
    .load()

# Декодування значень JSON з Kafka
streaming_df = kafka_stream_df \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*") \
    .withColumn("timestamp", current_timestamp())

# Запуск стрімінгу (імітуючи дані, що надходять в режимі реального часу)
streaming_query = streaming_df.writeStream \
    .format("memory") \
    .queryName("competition_stream") \
    .outputMode("append") \
    .start()

In [6]:
# Перетворення статичних даних в Streaming DataFrame

streaming_df = kafka_stream_df \
    .withColumn("timestamp", current_timestamp())

# Імітація стримінгу (дані надходять раз в 5 секунд)
streaming_query = streaming_df.writeStream \
    .format("memory") \
    .queryName("competition_stream") \
    .outputMode("append") \
    .start()

In [7]:
# Функція для обробки кожного мікробатчу (forEachBatch в завданні)

from pyspark.sql.functions import col, avg, current_timestamp

def process_batch(batch_df, batch_id):
    joined_df = batch_df.join(athlete_bio_df, 'athlete_id', 'inner')

    agg_df = joined_df.groupBy('sport', 'medal', 'sex', 'country_noc') \
        .agg(avg('height').alias('avg_height'), avg('weight').alias('avg_weight')) \
        .withColumn('calculated_at', current_timestamp())

    agg_df.show(truncate=False)

# Використовуємо існуючий streaming_df напряму
stream_query = streaming_df.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("update") \
    .start()

In [8]:
# Зчитування результатів з бази даних назад у DataFrame PySpark

jdbc_url = "jdbc:mysql://217.61.57.46:3306/olympic_dataset"
jdbc_table = "athlete_enriched_agg"
jdbc_user = "neo_data_admin"
jdbc_password = "Proyahaxuqithab9oplp"

result_df = spark.read.format('jdbc').options(
    url=jdbc_url,
    driver='com.mysql.cj.jdbc.Driver',
    dbtable=jdbc_table,
    user=jdbc_user,
    password=jdbc_password).load()

result_df.show()

+-------------+------+----+-----------+------------------+------------------+-------------------+
|        sport| medal| sex|country_noc|        avg_height|        avg_weight|          timestamp|
+-------------+------+----+-----------+------------------+------------------+-------------------+
|     Swimming|   nan|Male|        ANZ|             170.0|              65.0|2024-12-18 01:39:48|
|       Rowing|   nan|Male|        ANZ|             189.0|              85.0|2024-12-18 01:39:48|
|    Athletics|Bronze|Male|        ANZ|             184.0|              76.0|2024-12-18 01:39:48|
|     Swimming|Silver|Male|        ANZ|             170.0|              65.0|2024-12-18 01:39:48|
|     Swimming|Bronze|Male|        ANZ|             170.0|              65.0|2024-12-18 01:39:48|
|        Rugby|  Gold|Male|        ANZ|             175.0|              75.0|2024-12-18 01:39:48|
|    Athletics|   nan|Male|        ANZ|170.35294117647058| 65.05882352941177|2024-12-18 01:39:48|
|       Tennis|Bronz

In [9]:
# чи перетворення на Pandas DataFrame
pdf = result_df.toPandas()
pdf.head(20)

,sport,medal,sex,country_noc,avg_height,avg_weight,timestamp
0,Swimming,nan,Male,ANZ,170.000000,65.000000,2024-12-18 01:39:48
1,Rowing,nan,Male,ANZ,189.000000,85.000000,2024-12-18 01:39:48
2,Athletics,Bronze,Male,ANZ,184.000000,76.000000,2024-12-18 01:39:48
3,Swimming,Silver,Male,ANZ,170.000000,65.000000,2024-12-18 01:39:48
4,Swimming,Bronze,Male,ANZ,170.000000,65.000000,2024-12-18 01:39:48
5,Rugby,Gold,Male,ANZ,175.000000,75.000000,2024-12-18 01:39:48
6,Athletics,nan,Male,ANZ,170.352941,65.058824,2024-12-18 01:39:48
7,Tennis,Bronze,Male,ANZ,188.000000,84.000000,2024-12-18 01:39:48
8,Boxing,nan,Male,ANZ,167.000000,62.000000,2024-12-18 01:39:48
9,Athletics,nan,Male,BAN,169.782609,63.391304,2024-12-18 01:40:10


In [10]:
# Зупинка стрімінгу після виконання роботи

streaming_query.stop()
stream_query.stop()
spark.stop()